In [1]:
import pandas as pd
import csv

import os, os.path
import sys

# to move files from one directory to another
import shutil

In [2]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
plots_dir = "/Users/hn/Documents/01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/"

In [3]:
# choices_set_1 = pd.read_excel(io=param_dir + "set1_PerryandCo.xlsx", sheet_name=0)
# response_set_1 = pd.read_excel(io=param_dir + "Perry_and_Co_Responses_set_1.xlsx", sheet_name=0)

In [4]:
choices_set_1_xl = pd.ExcelFile(perry_dir + "set1_PerryandCo.xlsx")
choices_set_1_sheet_names = choices_set_1_xl.sheet_names  # see all sheet names

response_set_1_xl = pd.ExcelFile(perry_dir + "Perry_and_Co_Responses_set_1.xlsx")
response_set_1_sheet_names = response_set_1_xl.sheet_names  # see all sheet names


In [5]:
print (choices_set_1_sheet_names)
print (response_set_1_sheet_names)

['extended_1', 'extended_2', 'extended_3', 'extended_4', 'extended_5', 'extended_6']
['S1 - F6', 'S1 - F5', 'S1 - F4', 'S1 - F3', 'S1 - F2', 'S1 - F1']


In [6]:
a_choice_sheet = choices_set_1_xl.parse(choices_set_1_sheet_names[3])
sample_response = response_set_1_xl.parse(response_set_1_sheet_names[3])

In [7]:
print (list(a_choice_sheet.columns))

['Question_in_set', 'Question_overall', 'ID', 'NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB', 'latitude', 'longitude', 'CropTyp', 'Irrigtn', 'DataSrc', 'Acres', 'ExctAcr', 'LstSrvD', 'county']


## Count number of questions

In [8]:
question_count = 0

for a_choice_sheet in choices_set_1_sheet_names:
    
    # read a damn sheet
    a_choice_sheet = choices_set_1_xl.parse(a_choice_sheet)

    # add them to the damn list
    question_count += a_choice_sheet.shape[0]

print('There are [{ques_count}] questions.'.format(ques_count=question_count))

There are [294] questions.


In [9]:
# unwanted_opinions = ["kirtir@wsu.edu"]

In [10]:
sample_response.head(1)

,Timestamp,Email Address,"QUESTION 1 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.24608115,-119.94883182/@46.24608115,-119.94883182,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?,Comments,"QUESTION 2 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.97303283,-119.74909825/@46.97303283,-119.74909825,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.1,Comments.1,"QUESTION 3 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.97346577,-119.75313381/@46.97346577,-119.75313381,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.2,...,Comments.46,"QUESTION 48 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.532087,-120.74355913/@46.532087,-120.74355913,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.47,Comments.47,"QUESTION 49 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.53246669,-120.74090588/@46.53246669,-120.74090588,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.48,Comments.48,"QUESTION 50 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.48063414,-119.21048737/@46.48063414,-119.21048737,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.49,Comments.49
0,2022-02-02 11:41:50.037,andrew.mcguire@wsu.edu,Unsure,NaN,NaN,Single Crop,NaN,NaN,Single Crop,NaN,...,NaN,Single Crop,NaN,NaN,Single Crop,NaN,NaN,Double Crop,NaN,NaN


### Discussion Table

In [11]:
print(response_set_1_sheet_names[0])
print ("".join([s for s in response_set_1_sheet_names[0].split()[-1] if s.isdigit()]))

S1 - F6
6


In [12]:
## Define the damn output dataframe

discuss_table = pd.DataFrame(columns=['Discuss_Form', 'Discuss_Question', 'ID', 
                                      "Perry", "Andrew", "Tim", "Kirti"], 
                         index=range(question_count))
discuss_table.head(1)
curr_row = 0

extended_choices_discuss = pd.DataFrame()

###### populate the output datafrme

for response_sheet_name in response_set_1_sheet_names:
    
    # pick up the numeric part of the sheet names from google forms sheets
    # this is the form number as well.
    sheet_numeric_part = [s for s in response_sheet_name.split()[-1] if s.isdigit()]
    sheet_numeric_part = "".join(sheet_numeric_part)
    
    # form sheet names of choices excel sheets
    choice_sheet_name = "extended_" + sheet_numeric_part
    
    a_choice_sheet = choices_set_1_xl.parse(choice_sheet_name)
    a_response_sheet = response_set_1_xl.parse(response_sheet_name)
    
    no_people_in_sheet = a_response_sheet.shape[0]
    
    extended_choices_discuss = pd.concat([extended_choices_discuss, a_choice_sheet])
    #
    # drop unwanted opinions, i.e. keep experts opinions
    #
    # a_response_sheet = a_response_sheet[~(a_response_sheet["Email Address"].isin(unwanted_opinions))].copy()
    
    for a_col_name in a_response_sheet.columns:
        if ("Discuss" in a_col_name) and (a_response_sheet[a_col_name].isna().sum()<no_people_in_sheet):
            if a_col_name.split(".")[-1].isdigit():
                question_number = str(int(a_col_name.split(".")[-1])+1)
            else:
                question_number = "1"
            
            discuss_table.loc[curr_row, "Discuss_Form"] = sheet_numeric_part
            discuss_table.loc[curr_row, "Discuss_Question"] = question_number
#            discuss_table.loc[curr_row, "opinion_count"] = len(a_response_sheet[a_col_name].unique())
            discuss_table.loc[curr_row, "ID"] = a_choice_sheet.loc[int(question_number)-1, "ID"]
            
            discuss_table.loc[curr_row, "Perry"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "pbeale@agr.wa.gov"][a_col_name].values[0]
            
            discuss_table.loc[curr_row, "Andrew"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "andrew.mcguire@wsu.edu"][a_col_name].values[0]
            
            discuss_table.loc[curr_row, "Tim"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "twaters@wsu.edu"][a_col_name].values[0]
            
            if "kirtir@wsu.edu" in list(a_response_sheet["Email Address"]):
                discuss_table.loc[curr_row, "Kirti"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "kirtir@wsu.edu"][a_col_name].values[0]
            
            curr_row += 1
            
# drop rows where all row is NaNs
discuss_table = discuss_table.dropna(axis=0, how='all')

### Replace NaN with "No"
discuss_table = discuss_table.fillna("No")
discuss_table_extended = pd.merge(discuss_table, extended_choices_discuss, on=['ID'], how='left')
discuss_table_extended.shape

(32, 21)

### Comments

In [15]:
## Define the damn output dataframe

comment_table = pd.DataFrame(columns=['Comment_Form', 'Comment_Question', 'ID', 
                                      "PerryComment", "AndrewComment", "TimComment", "KirtiComment"], 
                         index=range(question_count))
comment_table.head(1)
curr_row = 0

extended_choices_comment = pd.DataFrame()

###### populate the output datafrme

for response_sheet_name in response_set_1_sheet_names:
    
    # pick up the numeric part of the sheet names from google forms sheets
    # this is the form number as well.
    sheet_numeric_part = [s for s in response_sheet_name.split()[-1] if s.isdigit()]
    sheet_numeric_part = "".join(sheet_numeric_part)
    
    # form sheet names of choices excel sheets
    choice_sheet_name = "extended_" + sheet_numeric_part
    
    a_choice_sheet = choices_set_1_xl.parse(choice_sheet_name)
    a_response_sheet = response_set_1_xl.parse(response_sheet_name)
    
    no_people_in_sheet = a_response_sheet.shape[0]
    
    extended_choices_comment = pd.concat([extended_choices_comment, a_choice_sheet])
    #
    # drop unwanted opinions, i.e. keep experts opinions
    #
    # a_response_sheet = a_response_sheet[~(a_response_sheet["Email Address"].isin(unwanted_opinions))].copy()
    
    for a_col_name in a_response_sheet.columns:
        if ("Comment" in a_col_name) and (a_response_sheet[a_col_name].isna().sum()<no_people_in_sheet):
            if a_col_name.split(".")[-1].isdigit():
                question_number = str(int(a_col_name.split(".")[-1])+1)
            else:
                question_number = "1"
            
            comment_table.loc[curr_row, "Comment_Form"] = sheet_numeric_part
            comment_table.loc[curr_row, "Comment_Question"] = question_number

            
            comment_table.loc[curr_row, "ID"] = a_choice_sheet.loc[int(question_number)-1, "ID"]
            
            comment_table.loc[curr_row, "PerryComment"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "pbeale@agr.wa.gov"][a_col_name].values[0]
            
            comment_table.loc[curr_row, "AndrewComment"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "andrew.mcguire@wsu.edu"][a_col_name].values[0]
            
            comment_table.loc[curr_row, "TimComment"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "twaters@wsu.edu"][a_col_name].values[0]
            
            if "kirtir@wsu.edu" in list(a_response_sheet["Email Address"]):
                comment_table.loc[curr_row, "KirtiComment"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "kirtir@wsu.edu"][a_col_name].values[0]
            
            curr_row += 1
            
# drop rows where all row is NaNs
comment_table = comment_table.dropna(axis=0, how='all')

### Replace NaN with "No"
comment_table = comment_table.fillna("No")

comment_table_extended = pd.merge(comment_table, extended_choices_comment, on=['ID'], how='left')
comment_table_extended.shape

(34, 21)

In [16]:
output_together = pd.merge(discuss_table_extended, comment_table_extended, on=['ID'], how='outer')
output_together = output_together.fillna("No")
output_together.shape

(50, 41)

In [17]:
output_together.tail(2)

,Discuss_Form,Discuss_Question,ID,Perry,Andrew,Tim,Kirti,Question_in_set_x,Question_overall_x,NDVI_TS_Name_x,...,TOA_RGB_y,latitude_y,longitude_y,CropTyp_y,Irrigtn_y,DataSrc_y,Acres_y,ExctAcr_y,LstSrvD_y,county_y
48,No,No,108842_WSDA_SF_2017,No,No,No,No,No,No,No,...,108842_WSDA_SF_2017_47.090746_-119.81387885_TO...,47.090746,-119.813879,wheat,center pivot,wsda,32.0,32.301584,2017/10/16,Grant
49,No,No,109375_WSDA_SF_2017,No,No,No,No,No,No,No,...,109375_WSDA_SF_2017_47.13175543_-119.62366484_...,47.131755,-119.623665,wheat fallow,center pivot,wsda,49.0,48.55717,2017/08/28,Grant


In [18]:
print (comment_table.ID.isin(discuss_table.ID).sum())
print (discuss_table.ID.isin(comment_table.ID).sum())

16
16


## Fix the Form and Question numbers

In [19]:
for row in range(output_together.shape[0]):
    if output_together.loc[row, "Discuss_Form"]=="No":
        output_together.loc[row, "Discuss_Form"] = output_together.loc[row, "Comment_Form"]
        output_together.loc[row, "Discuss_Question"] = output_together.loc[row, "Comment_Question"]
        output_together.loc[row, "NDVI_TS_Name_x"] = output_together.loc[row, "NDVI_TS_Name_y"]
        output_together.loc[row, "CropTyp_x"] = output_together.loc[row, "CropTyp_y"]

In [20]:
output_together.drop(labels=['corrected_RGB_y', 'TOA_RGB_y', 'latitude_y', 'longitude_y',
                             'Irrigtn_y', 'DataSrc_y', 'Acres_y', 'ExctAcr_y',
                             'LstSrvD_y', 'county_y', 'Question_in_set_y', 'Question_overall_y',
                             'county_x', 'Irrigtn_x',  'corrected_RGB_x', 'TOA_RGB_x', 'latitude_x', 
                             'longitude_x', 'DataSrc_x', 'Acres_x', 'ExctAcr_x', 'LstSrvD_x',
                             'Question_in_set_x', 'Question_overall_x'], 
                         axis='columns', inplace=True)

In [21]:
output_together.columns

Index(['Discuss_Form', 'Discuss_Question', 'ID', 'Perry', 'Andrew', 'Tim',
       'Kirti', 'NDVI_TS_Name_x', 'CropTyp_x', 'Comment_Form',
       'Comment_Question', 'PerryComment', 'AndrewComment', 'TimComment',
       'KirtiComment', 'NDVI_TS_Name_y', 'CropTyp_y'],
      dtype='object')

In [22]:
sorted(output_together.Discuss_Form.unique())

['1', '2', '3', '4', '5', '6']

# Disagreements start here:

In [23]:
choices_set_1_xl = pd.ExcelFile(perry_dir + "set1_PerryandCo.xlsx")
choices_set_1_sheet_names = choices_set_1_xl.sheet_names  # see all sheet names

response_set_1_xl = pd.ExcelFile(perry_dir + "Perry_and_Co_Responses_set_1.xlsx")
response_set_1_sheet_names = response_set_1_xl.sheet_names  # see all sheet names


"""   Count number of questions
"""

question_count = 0

for a_choice_sheet in choices_set_1_sheet_names:
    
    # read a damn sheet
    a_choice_sheet = choices_set_1_xl.parse(a_choice_sheet)

    # add them to the damn list
    question_count += a_choice_sheet.shape[0]

print('There are [{ques_count}] questions.'.format(ques_count=question_count))

There are [294] questions.


In [24]:
## Define the damn output dataframe
unwanted_opinions = ["kirtir@wsu.edu"]

output_df = pd.DataFrame(columns=['Disagreement_form', 'disagreement_Question', \
                                  'opinion_count', 'ID', "PerrysVote", "AndrewsVote", "TimsVote"], 
                         index=range(question_count))
output_df.head(1)
curr_row = 0

extended_choices = pd.DataFrame()

###### populate the output datafrme

for response_sheet_name in response_set_1_sheet_names:
    
    # pick up the numeric part of the sheet names from google Disagreement_forms sheets
    # this is the Disagreement_form number as well.
    sheet_numeric_part = [s for s in response_sheet_name.split()[-1] if s.isdigit()]
    sheet_numeric_part = "".join(sheet_numeric_part)
    
    # Disagreement_form sheet names of choices excel sheets
    choice_sheet_name = "extended_" + sheet_numeric_part
    
    a_choice_sheet = choices_set_1_xl.parse(choice_sheet_name)
    a_response_sheet = response_set_1_xl.parse(response_sheet_name)
    
    extended_choices = pd.concat([extended_choices, a_choice_sheet])
    #
    # drop unwanted opinions, i.e. keep experts opinions
    #
    a_response_sheet = a_response_sheet[~(a_response_sheet["Email Address"].isin(unwanted_opinions))].copy()
    
    for a_col_name in a_response_sheet.columns:
        if "QUESTION" in a_col_name:
            question_number = a_col_name.split()[1]
            
            output_df.loc[curr_row, "Disagreement_form"] = sheet_numeric_part
            output_df.loc[curr_row, "disagreement_Question"] = question_number
            output_df.loc[curr_row, "opinion_count"] = len(a_response_sheet[a_col_name].unique())
            output_df.loc[curr_row, "ID"] = a_choice_sheet.loc[int(question_number)-1, "ID"]
            
             
            output_df.loc[curr_row, "PerrysVote"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "pbeale@agr.wa.gov"][a_col_name].values[0]
            
            output_df.loc[curr_row, "AndrewsVote"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "andrew.mcguire@wsu.edu"][a_col_name].values[0]
            
            output_df.loc[curr_row, "TimsVote"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "twaters@wsu.edu"][a_col_name].values[0]
            
            curr_row += 1

extended_output = pd.merge(output_df, extended_choices, on=['ID'], how='left')
extended_output.head(2)

disagreement_table = extended_output[extended_output.opinion_count != 1].copy()
disagreement_table = disagreement_table[disagreement_table.CropTyp != "grass hay"].copy()
disagreement_table.reset_index(inplace=True, drop=True)

disagreement_table.drop(labels=['TOA_RGB', 'latitude', 'longitude', 'Irrigtn', 'DataSrc', 'Acres',
                                'ExctAcr', 'LstSrvD', 'county', 'Question_in_set', 'Question_overall',
                                'corrected_RGB'], 
                         axis='columns', inplace=True)


disagreement_table.head(1)

,Disagreement_form,disagreement_Question,opinion_count,ID,PerrysVote,AndrewsVote,TimsVote,NDVI_TS_Name,CropTyp
0,6,8,3,107177_WSDA_SF_2018,Double Crop,Either double or mustard crop,Single Crop,FranklinYakima2018_107177_WSDA_SF_2018.png,grass seed


In [25]:
sorted(disagreement_table.CropTyp.unique())

['barley',
 'barley hay',
 'bean, dry',
 'bean, green',
 'bluegrass seed',
 'buckwheat',
 'canola',
 'carrot',
 'carrot seed',
 'corn seed',
 'corn, field',
 'corn, sweet',
 'grass seed',
 'market crops',
 'mint',
 'oat hay',
 'onion',
 'pea seed',
 'pea, dry',
 'pea, green',
 'potato',
 'triticale',
 'triticale hay',
 'wheat',
 'wheat fallow',
 'yellow mustard']

In [ ]:
three_together = pd.merge(output_together, disagreement_table, on=['ID'], how='outer')
three_together = three_together.fillna("No")
three_together.head(5)

In [ ]:
three_together.head(2)

In [ ]:
sorted(extended_output.columns)

## Fix the Form and Question numbers

In [ ]:
for row in range(three_together.shape[0]):
    if three_together.loc[row, "Discuss_Form"]=="No":
        cc = extended_output[extended_output.ID == three_together.loc[row, "ID"]]
        three_together.loc[row, "Discuss_Form"] = cc.iloc[0]['Disagreement_form']
    
    if three_together.loc[row, "Discuss_Question"] == "No":
        cc = extended_output[extended_output.ID == three_together.loc[row, "ID"]]
        three_together.loc[row, "Discuss_Question"] = cc.iloc[0]['disagreement_Question']
            
    if three_together.loc[row, "NDVI_TS_Name_x"] == "No":
        cc = extended_output[extended_output.ID == three_together.loc[row, "ID"]]
        three_together.loc[row, "NDVI_TS_Name_x"] = cc.iloc[0]['NDVI_TS_Name']
        
    if three_together.loc[row, "CropTyp_x"] == "No": 
        cc = extended_output[extended_output.ID == three_together.loc[row, "ID"]]
        three_together.loc[row, "CropTyp_x"] = cc.iloc[0]['CropTyp']


In [ ]:
print (sorted(three_together.Discuss_Form.unique()))
print (sorted(three_together.Discuss_Question.unique()))

### Create a long PDF of fields where discussion is requested

In [ ]:
import subprocess
os.getcwd()

In [ ]:
three_together.sort_values(by=['CropTyp_x'],
                           ascending = [True],
                           inplace=True)
three_together.reset_index(inplace=True, drop=True)
three_together.head(2)

os.chdir('/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/set_1_PDFs/')

with open("set_1_3in1_Discussion_Comments_Disagreements.tex","w") as file:
    file.write("\\documentclass{article}\n")

    file.write("\\usepackage[osf,sc]{mathpazo}\n")
    file.write("\\usepackage{color}\n")
    file.write("\\definecolor{mgreen}{RGB}{25,147,100}\n")

    file.write("\\usepackage{mathtools,hyperref}\n")
    file.write("\\hypersetup{\n")
    file.write("      colorlinks=true,\n")
    file.write("      linkcolor=cyan,\n")
    file.write("      filecolor=cyan,\n")
    file.write("      urlcolor=mgreen,\n")
    file.write("      citecolor=cyan}\n")
    

    file.write("\\usepackage[margin=1in]{geometry}\n")
    file.write("\\usepackage[T1]{fontenc}\n")
    
    file.write("\\usepackage{caption}\n")
    file.write("\\captionsetup{justification= raggedright, singlelinecheck = false}\n")

    file.write("\\title{\\bf Set 1 - Discussion, Comments, and Disgreements}\n")
    file.write("\\author{}\n")
    file.write("\\date{}\n")


    file.write("\\begin{document}\n")
    file.write("\\maketitle\n")

    # file.write("\\section{Discussion, Comments, and Disgreements}\n")
    
    file.write("\n")
    file.write("\n")
    
    for a_row in range(three_together.shape[0]):
        curr_row = three_together.loc[a_row]
        # print (curr_row)
        TS_file_name = curr_row["NDVI_TS_Name_x"]
        file.write("\\begin{figure*}[ht]\n")
        file.write("\\centering\n")
        file.write("\\includegraphics[width=1\\textwidth]{/Users/hn/Documents/" + \
                     "01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/" + \
                         TS_file_name + "}\n")
        file.write("\\caption[]{" + \
                   "\\textbf{\\color{red}{" + curr_row["CropTyp_x"] + "}}" + \
                   ", ID: " + curr_row["ID"].replace("_", "\_") + \
                   ", Form: " + curr_row["Discuss_Form"] + \
                   ", Question: " + curr_row["Discuss_Question"] + "." + \
                   "}\n")
        file.write("\\label{fig:figure" + str(a_row) + "}\n")
        file.write("\\end{figure*}\n")
        file.write("\n")
        
        if str(curr_row["Perry"])==str(curr_row["Andrew"])==str(curr_row["Tim"])==str(curr_row["Kirti"])=="No":
            file.write("\\noindent \\textbf{Discuss? {\color{mgreen}{No}}}" + "\\\\\n")
        else:
            file.write("\\noindent \\textbf{Discuss? {\color{red}{Yes}}}" + "\\\\\n")
            
        file.write("\\noindent Perry: ~~~~" + str(curr_row["Perry"]) + "\\\\\n")
        file.write("Andrew: " + str(curr_row["Andrew"]) + "\\\\\n")
        file.write("Tim: ~~~~~~~" + str(curr_row["Tim"]) +  "\\\\\n")
        file.write("Kirti: ~~~~~~" + str(curr_row["Kirti"]) + "\\\\\n")
        
        file.write("\n")
        file.write("\n")
        if str(curr_row["PerrysVote"])==str(curr_row["AndrewsVote"])==str(curr_row["TimsVote"]) == "No":
            aaa = extended_output[extended_output.ID == curr_row.ID].copy()
            file.write("\\noindent \\textbf{Disagreement? {\color{mgreen}{No}}}" + "\\\\\n")
            file.write("\\noindent Perry's Vote:~~~~ " + aaa.iloc[0]["PerrysVote"] + "\\\\\n")
            file.write("Andrew's Vote: " + aaa.iloc[0]["AndrewsVote"]  + "\\\\\n")
            file.write("Tim's Vote: ~~~~~ " + aaa.iloc[0]["TimsVote"]  +  "\\\\\n")        
        else:
            file.write("\\noindent \\textbf{Disagreement? {\color{red}{Yes}}}" + "\\\\\n")
            file.write("\\noindent Perry's Vote:~~~~ " + str(curr_row["PerrysVote"]) + "\\\\\n")
            file.write("Andrew's Vote: " + str(curr_row["AndrewsVote"]) + "\\\\\n")
            file.write("Tim's Vote: ~~~~~ " + str(curr_row["TimsVote"]) +  "\\\\\n")        
        
        file.write("\n")
        file.write("\n")
        file.write("\\noindent \\textbf{Comments}" + "\\\\\n")
        file.write("\\noindent Perry: ~~~~" + str(curr_row["PerryComment"]) + "\\\\\n")
        file.write("Andrew: " + str(curr_row["AndrewComment"]) + "\\\\\n")
        file.write("Tim: ~~~~~~~" + str(curr_row["TimComment"]) +  "\\\\\n")
        file.write("Kirti: ~~~~~~" + str(curr_row["KirtiComment"]) + "\\\\\n")
        
                
        file.write("\\clearpage\n")

    file.write("\\end{document}\n")

In [ ]:
curr_row = three_together.loc[a_row]

In [ ]:
curr_row

In [ ]:
str(curr_row["TimsVote"])

In [ ]:
three_together